In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [4]:
genre = pd.read_csv('/content/drive/My Drive/FinalDataProject/Machine/numerical_machine2.csv')
genre

,Genre,mfcc(0),mfcc(1),mfcc(2),mfcc(3),mfcc(4),mfcc(5),mfcc(6),mfcc(7),mfcc(8),mfcc(9),mfcc(10),mfcc(11),mfcc(12),mfcc(13),mfcc(14),mfcc(15),mfcc(16),mfcc(17),mfcc(18),mfcc(19),mfcc(20),mfcc(21),mfcc(22),mfcc(23),mfcc(24),mfcc(25),mfcc(26),mfcc(27),mfcc(28),mfcc(29),Spectral Centroid(0),RSME(0),Spectral Rolloff(0),Spectral Bandwidth(0),Zero-Crossing Rate(0),Chroma Feature(0),Spectral Centroid(1),RSME(1),Spectral Rolloff(1),...,Spectral Rolloff(23),Spectral Bandwidth(23),Zero-Crossing Rate(23),Chroma Feature(23),Spectral Centroid(24),RSME(24),Spectral Rolloff(24),Spectral Bandwidth(24),Zero-Crossing Rate(24),Chroma Feature(24),Spectral Centroid(25),RSME(25),Spectral Rolloff(25),Spectral Bandwidth(25),Zero-Crossing Rate(25),Chroma Feature(25),Spectral Centroid(26),RSME(26),Spectral Rolloff(26),Spectral Bandwidth(26),Zero-Crossing Rate(26),Chroma Feature(26),Spectral Centroid(27),RSME(27),Spectral Rolloff(27),Spectral Bandwidth(27),Zero-Crossing Rate(27),Chroma Feature(27),Spectral Centroid(28),RSME(28),Spectral Rolloff(28),Spectral Bandwidth(28),Zero-Crossing Rate(28),Chroma Feature(28),Spectral Centroid(29),RSME(29),Spectral Rolloff(29),Spectral Bandwidth(29),Zero-Crossing Rate(29),Chroma Feature(29)
0,blues,-1.080816,2.402548,2.447845,-2.650289,-1.626252,-2.990681,-5.615472,-4.249632,1.396113,5.773273,-0.170325,-1.023890,-2.232717,-1.980449,-1.205221,0.693168,1.117244,-3.761428,-0.673625,-4.207499,-3.987191,-6.407551,1.612845,-1.185128,-0.607955,-4.636970,-3.990787,-1.013927,-5.099481,-2.020702,1600.108227,0.068621,3125.495494,1768.178823,0.071844,0.366233,2169.54364,0.072983,4673.683860,...,4018.389338,2170.258859,0.088712,0.356626,1833.694183,0.072447,3601.917614,2004.806190,0.075872,0.490950,2177.208805,0.074474,4522.951438,2277.413443,0.085372,0.352899,1840.786497,0.080259,3891.392378,2080.568947,0.066850,0.333888,2019.606758,0.063985,4030.868808,2083.296498,0.093428,0.343766,1918.636318,0.067129,3923.202792,2075.408600,0.077981,0.293308,1929.611914,0.075155,3934.948176,2085.801994,0.079978,0.319906
1,blues,-7.706903,-12.349855,-10.180769,-8.752623,-9.936936,-9.948211,-9.690319,-5.899460,-10.677017,-11.491119,-10.189684,-8.855388,-10.111384,-9.833827,-12.207047,-9.036697,-3.832873,-10.028079,-8.660783,-8.067840,-9.294783,-9.966580,-8.525799,-13.828783,-7.417404,-11.106751,-8.492090,-8.880114,-11.921321,-5.806519,1600.108227,0.068621,3125.495494,1768.178823,0.071844,0.366233,2169.54364,0.072983,4673.683860,...,4018.389338,2170.258859,0.088712,0.356626,1833.694183,0.072447,3601.917614,2004.806190,0.075872,0.490950,2177.208805,0.074474,4522.951438,2277.413443,0.085372,0.352899,1840.786497,0.080259,3891.392378,2080.568947,0.066850,0.333888,2019.606758,0.063985,4030.868808,2083.296498,0.093428,0.343766,1918.636318,0.067129,3923.202792,2075.408600,0.077981,0.293308,1929.611914,0.075155,3934.948176,2085.801994,0.079978,0.319906
2,blues,11.065468,10.212138,7.413080,9.423204,6.787949,5.061450,2.695190,4.992860,5.641868,6.743513,5.775306,5.835487,8.090005,7.838480,5.904506,5.732425,7.445722,4.650060,0.364224,5.940204,8.180415,9.759674,5.728453,9.910507,6.010060,7.298434,4.920581,8.779825,5.741844,7.158023,1600.108227,0.068621,3125.495494,1768.178823,0.071844,0.366233,2169.54364,0.072983,4673.683860,...,4018.389338,2170.258859,0.088712,0.356626,1833.694183,0.072447,3601.917614,2004.806190,0.075872,0.490950,2177.208805,0.074474,4522.951438,2277.413443,0.085372,0.352899,1840.786497,0.080259,3891.392378,2080.568947,0.066850,0.333888,2019.606758,0.063985,4030.868808,2083.296498,0.093428,0.343766,1918.636318,0.067129,3923.202792,2075.408600,0.077981,0.293308,1929.611914,0.075155,3934.948176,2085.801994,0.079978,0.319906
3,blues,-3.334484,-7.000736,-1.607500,-2.864413,-7.559600,-3.691948,-0.329426,-5.618731,1.696022,-0.701657,-8.466679,-5.054170,-0.312030,-8.425481,-10.371735,-3.673921,-10.473954,-11.042602,1.526414,-4.568877,-3.232481,-3.953928,-7.513575,0.052442,-4.963974,-4.622189,-0.962560,-1.588208,-6.786798,1.039369,1600.108227,0.068

## Data Pre-Processing

In [5]:
#define dependent variable (y) and all independent variables(x)
X = genre.drop("Genre", axis=1)
y = genre["Genre"]
print(X.shape, y.shape)

(1000, 210) (1000,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
#split data into 80% train data and 20% test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
#Scale X data to fit model for efficiency and performance
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set string to integer
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding(integers to binary code)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [11]:
#Deep learning dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

In [13]:
# Create model and add layers, 3 layers each with 75 nodes, output set to 10 because we have 10 genres
model = Sequential()
model.add(Dense(units=75, activation='relu', input_dim=210))
model.add(Dense(units=75, activation='relu'))
model.add(Dense(units=75, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
#display shape/summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 75)                15825     
_________________________________________________________________
dense_1 (Dense)              (None, 75)                5700      
_________________________________________________________________
dense_2 (Dense)              (None, 75)                5700      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                760       
Total params: 27,985
Trainable params: 27,985
Non-trainable params: 0
_________________________________________________________________


In [16]:
#train our model for 120 cycles
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=120,
    shuffle=True,
    verbose=2
)

Epoch 1/120
24/24 - 0s - loss: 1.8965 - accuracy: 0.6307
Epoch 2/120
24/24 - 0s - loss: 0.8251 - accuracy: 0.9907
Epoch 3/120
24/24 - 0s - loss: 0.1361 - accuracy: 1.0000
Epoch 4/120
24/24 - 0s - loss: 0.0270 - accuracy: 1.0000
Epoch 5/120
24/24 - 0s - loss: 0.0113 - accuracy: 1.0000
Epoch 6/120
24/24 - 0s - loss: 0.0066 - accuracy: 1.0000
Epoch 7/120
24/24 - 0s - loss: 0.0044 - accuracy: 1.0000
Epoch 8/120
24/24 - 0s - loss: 0.0030 - accuracy: 1.0000
Epoch 9/120
24/24 - 0s - loss: 0.0022 - accuracy: 1.0000
Epoch 10/120
24/24 - 0s - loss: 0.0017 - accuracy: 1.0000
Epoch 11/120
24/24 - 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 12/120
24/24 - 0s - loss: 0.0011 - accuracy: 1.0000
Epoch 13/120
24/24 - 0s - loss: 8.9780e-04 - accuracy: 1.0000
Epoch 14/120
24/24 - 0s - loss: 7.4931e-04 - accuracy: 1.0000
Epoch 15/120
24/24 - 0s - loss: 6.3836e-04 - accuracy: 1.0000
Epoch 16/120
24/24 - 0s - loss: 5.5073e-04 - accuracy: 1.0000
Epoch 17/120
24/24 - 0s - loss: 4.7664e-04 - accuracy: 1.0000
Epo

In [17]:
# Save the model
model.save("genre_model_perf.h5")

## Quantify our Trained Model

In [18]:
#test the trained model against our test dataset
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 1.2815e-05 - accuracy: 1.0000
Normal Neural Network - Loss: 1.2815003174182493e-05, Accuracy: 1.0


## Make Predictions

In [19]:
#putting machines predictions and test values into variables
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
#printing machine predictions and test values to compare
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['hiphop' 'reggae' 'heavy metal' 'disco' 'country' 'rock' 'country'
 'reggae' 'disco' 'blues']
Actual Labels: ['hiphop', 'reggae', 'heavy metal', 'disco', 'country', 'rock', 'country', 'reggae', 'disco', 'blues']


In [21]:
# # Load the model
from tensorflow.keras.models import load_model
genre_model = load_model("genre_model_perf.h5")

In [22]:
#display info/stats of saved model
model_loss, model_accuracy = genre_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Music Genre Classification:Deep Learning Model - Loss: {model_loss}, Accuracy: {model_accuracy}")

8/8 - 0s - loss: 1.2815e-05 - accuracy: 1.0000
Music Genre Classification:Deep Learning Model - Loss: 1.2815003174182493e-05, Accuracy: 1.0
